# Video Downsampling with VideoDownsamplingStep

This Jupyter Notebook demonstrates how to use the `VideoDownsamplingStep` class from the `ai_graph` Python package to downsample a video to a specified frames-per-second (FPS), resolution, and format. The code uses FFmpeg for video processing and supports GPU acceleration if available. We'll also download a sample video using `wget` to test the downsampling process.

## Prerequisites
- **FFmpeg**: Ensure FFmpeg is installed and accessible in your system's PATH.
- **Python Libraries**: Install required libraries (`ai_graph`, `opencv-python`, `torch`).
- **wget**: For downloading the sample video.

Let's walk through the process step-by-step.

## Step 1: Install Required Libraries

First, let's install the necessary Python libraries. Run the following cell to install `ai_graph`, `opencv-python`, and `torch` if they are not already installed.

In [ ]:
!pip install ai_graph opencv-python torch

## Step 2: Download a Sample Video

We'll use `wget` to download a sample video from a public source. For this example, we'll download a short, royalty-free video from [Pexels](https://www.pexels.com). The video will be saved as `sample_video.mp4`.

In [ ]:
import os

# Download a sample video using wget
video_url = "https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_50mb.mp4"
video_file_name = "sample_video.mp4"
!wget -O {video_file_name} {video_url}

# Verify the video was downloaded
if os.path.isfile(video_file_name):
    print(f"Sample video downloaded successfully: {video_file_name}")
else:
    print("Failed to download the sample video.")

--2025-08-18 17:11:46--  https://www.pexels.com/video/857251/download/
Resolving www.pexels.com (www.pexels.com)... 10.10.34.36
Connecting to www.pexels.com (www.pexels.com)|10.10.34.36|:443... failed: Unknown error.
Retrying.

--2025-08-18 17:12:09--  (try: 2)  https://www.pexels.com/video/857251/download/
Connecting to www.pexels.com (www.pexels.com)|10.10.34.36|:443... failed: Unknown error.
Retrying.

--2025-08-18 17:12:32--  (try: 3)  https://www.pexels.com/video/857251/download/
Connecting to www.pexels.com (www.pexels.com)|10.10.34.36|:443... failed: Unknown error.
Retrying.

--2025-08-18 17:12:56--  (try: 4)  https://www.pexels.com/video/857251/download/
Connecting to www.pexels.com (www.pexels.com)|10.10.34.36|:443... failed: Unknown error.
Retrying.

--2025-08-18 17:13:21--  (try: 5)  https://www.pexels.com/video/857251/download/
Connecting to www.pexels.com (www.pexels.com)|10.10.34.36|:443... failed: Unknown error.
Retrying.

--2025-08-18 17:13:47--  (try: 6)  https://www.p

## Step 3: Import the VideoDownsamplingStep

The `VideoDownsamplingStep` class is part of the `ai_graph` package. It handles video downsampling by adjusting FPS, resolution, and/or format using FFmpeg. Below, we import the necessary classes from `ai_graph`.

In [ ]:
from ai_graph.pipeline.base import Pipeline
from ai_graph.step.video.video_downsampling import VideoDownsamplingStep
import logging
import os
import cv2

## Step 4: Set Up Logging

To see detailed logs during the downsampling process, let's configure the logging module.

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Step 5: Run the Video Downsampling Pipeline

Now, let's create a pipeline, add the `VideoDownsamplingStep`, and process the downloaded sample video. We'll downsample it to 1 FPS, 640x360 resolution, and MP4 format.

In [ ]:
# Define the path to the input video
video_path = os.path.abspath(video_file_name)

# Define output parameters
output_fps = 1
output_resolution = "640x360"
output_format = "mp4"

# Create a pipeline
pipeline = Pipeline(name="VideoDownsamplingPipeline")

# Add the VideoDownsamplingStep to the pipeline
pipeline.add_step(
    VideoDownsamplingStep(
        output_fps=output_fps,
        output_resolution=output_resolution,
        output_format=output_format,
        name="DownsampleVideo"
    )
)

# Prepare the input data for the pipeline
input_data = {"video_path": video_path}

print(f"Starting video downsampling for: {video_path}")
print(f"Target FPS: {output_fps}, Resolution: {output_resolution}, Format: {output_format}")

# Process the video through the pipeline
result = pipeline.process(input_data)

print("\nVideo downsampling pipeline completed.")
print("Result:")
print(result)

# Check the downsampled video path
if "video_path" in result:
    print(f"Downsampled video saved to: {result['video_path']}")
else:
    print("Downsampled video path not found in result.")

## Step 6: Verify the Output

After running the pipeline, the downsampled video should be saved in the same directory as the input video with a `_downsampled` suffix (e.g., `sample_video_downsampled.mp4`). You can verify the output file exists and check its properties using `cv2`.

In [ ]:
# Verify the output video
if "video_path" in result and os.path.isfile(result["video_path"]):
    cap = cv2.VideoCapture(result["video_path"])
    if cap.isOpened():
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"Output video FPS: {fps}")
        print(f"Output video resolution: {width}x{height}")
        cap.release()
    else:
        print("Failed to open the downsampled video.")
else:
    print("Output video file not found.")

## Explanation of the Code

- **VideoDownsamplingStep**: This class, part of the `ai_graph.step.video` module, inherits from `BasePipelineStep` and handles video downsampling. It takes parameters like `output_fps`, `output_resolution`, and `output_format` to specify the target video properties.
- **_process_step**: This method checks the input video, retrieves its original FPS using OpenCV, and decides whether downsampling is needed. It constructs the output filename and calls `_downsample_video`.
- **_downsample_video**: This method builds and executes an FFmpeg command to perform the downsampling. It supports GPU acceleration (using `h264_nvenc`) if a CUDA-enabled GPU is detected via `torch.cuda.is_available()`. Otherwise, it uses CPU-based encoding (`libx264` for MP4, `libvpx-vp9` for WebM).
- **Pipeline**: The `Pipeline` class from `ai_graph.pipeline.base` manages the execution of steps. In this case, it only contains the `VideoDownsamplingStep`.

## Notes
- Ensure FFmpeg is installed and accessible in your system's PATH.
- If you have a CUDA-enabled GPU, the code will attempt to use hardware acceleration, which can significantly speed up processing.
- The output video is saved with a `_downsampled` suffix in the same directory as the input video.
- If the original FPS matches the target FPS, the downsampling step is skipped to avoid unnecessary processing.